In [1]:
import numpy as np
import tensorflow as tf
import random as rn
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(1)
from keras import backend as K
#tf.compat.v1.set_random_seed(1)
#sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph())
#K.set_session(sess)
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.layers import Conv1D,MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import RMSprop
import keras.regularizers
import scipy
import math
import sys
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress
from scipy import interpolate
from scipy import signal
import pickle
import collections
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TerminateOnNaN
from video_process_utils import *
import statsmodels.api as sm

C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [2]:
target_col = 'GDI'

In [3]:
alldata_processed =\
    pd.read_csv("data/processed/alldata_processed_with_dev_residual.csv" )
alldata_processed['videoid'] = alldata_processed['videoid'].apply(lambda x: int(x))
alldata_processed['target_count'] = alldata_processed.groupby('videoid')[target_col].transform(lambda x: x.count())

In [4]:
alldata_processed = alldata_processed[alldata_processed[target_col].notnull()]
alldata_processed = alldata_processed[alldata_processed['target_count'] == 2]
ids_nonmissing_target = set(alldata_processed['videoid'].unique())

In [5]:
datasplit_df = pd.read_csv('./data/processed/train_test_valid_id_split.csv')
datasplit_df['videoid'] = datasplit_df['videoid'].apply(lambda x: int(x))
all_ids = set(datasplit_df['videoid']).intersection(ids_nonmissing_target)
train_ids = set(datasplit_df[datasplit_df['dataset'] == 'train']['videoid']).intersection(ids_nonmissing_target)
validation_ids = set(datasplit_df[datasplit_df['dataset'] == 'validation']['videoid']).intersection(ids_nonmissing_target)
test_ids = set(datasplit_df[datasplit_df['dataset'] == 'test']['videoid']).intersection(ids_nonmissing_target)

In [6]:
with open('./data/processed/all_processed_video_segments.pickle', 'rb') as handle:
    processed_video_segments = pickle.load(handle)

In [7]:
assert(np.sum(alldata_processed[target_col].isnull()) == 0)

In [8]:
x_columns_left = [2*LANK,2*LANK+1,2*LKNE,2*LKNE+1,
        2*LHIP,2*LHIP+1,2*LBTO,2*LBTO+1,50,52,54,56]
x_columns_right = [2*RANK,2*RANK+1,2*RKNE,2*RKNE+1,
        2*RHIP,2*RHIP+1,2*RBTO,2*RBTO+1,51,53,55,57]

target_dict_L = {}
target_dict_R = {}
for i in range(len(alldata_processed)):
    row = alldata_processed.iloc[i]
    if row['side'] == 'L':
        target_dict_L[row['videoid']] = row[target_col]
    if row['side'] == 'R':
        target_dict_R[row['videoid']] = row[target_col]

In [9]:
X = [t[2] for t in processed_video_segments if t[0] in all_ids]
X = np.stack(X)
X = np.concatenate([X[:,:,x_columns_left],X[:,:,x_columns_right]])

In [10]:
X_train = [t[2] for t in processed_video_segments if t[0] in train_ids]
X_train = np.stack(X_train)
X_train = np.concatenate([X_train[:,:,x_columns_left],X_train[:,:,x_columns_right]])

In [11]:
X_validation = [t[2] for t in processed_video_segments if t[0] in validation_ids]
X_validation = np.stack(X_validation)
X_validation = np.concatenate([X_validation[:,:,x_columns_left],X_validation[:,:,x_columns_right]])

In [12]:
y_train_L = [target_dict_L[t[0]] for t in processed_video_segments if t[0] in train_ids]
y_train_R = [target_dict_R[t[0]] for t in processed_video_segments if t[0] in train_ids]
y_train = np.array(y_train_L + y_train_R)

In [13]:
y_validation_L = [target_dict_L[t[0]] for t in processed_video_segments if t[0] in validation_ids]
y_validation_R = [target_dict_R[t[0]] for t in processed_video_segments if t[0] in validation_ids]
y_validation = np.array(y_validation_L + y_validation_R)

In [14]:
y_L = [target_dict_L[t[0]] for t in processed_video_segments if t[0] in all_ids]
y_R = [target_dict_R[t[0]] for t in processed_video_segments if t[0] in all_ids]
y = np.array(y_L + y_R)

In [15]:
videoid_count_dict = collections.Counter(np.array([t[0] for t in processed_video_segments]))

In [16]:
train_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in train_ids]
train_videoid_weights = train_videoid_weights + train_videoid_weights
train_videoid_weights = np.array(train_videoid_weights).reshape(-1,1)
validation_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in validation_ids]
validation_videoid_weights = validation_videoid_weights + validation_videoid_weights
validation_videoid_weights = np.array(validation_videoid_weights).reshape(-1,1)

In [17]:
target_min = np.min(y_train,axis=0)
target_range = np.max(y_train,axis=0) - np.min(y_train,axis=0)

In [18]:
y_train_scaled = ((y_train-target_min)/target_range).reshape(-1,1)
y_validation_scaled = ((y_validation-target_min)/target_range).reshape(-1,1)
y_validation_scaled = np.hstack([y_validation_scaled,validation_videoid_weights])
y_train_scaled = np.hstack([y_train_scaled,train_videoid_weights])
#c_i_factor is just a constant initially introduced to make the loss function more interpretable, but
#is probably not necessary if the model were to be retrained
c_i_factor = np.mean(np.vstack([train_videoid_weights,validation_videoid_weights])) 

In [19]:
vid_length = 124

In [21]:
def step_decay(initial_lrate,epochs_drop,drop_factor):
    def step_decay_fcn(epoch):
        return initial_lrate * math.pow(drop_factor, math.floor((1+epoch)/epochs_drop))
    return step_decay_fcn

epochs_drop,drop_factor = (10,0.8)
initial_lrate = 0.001
dropout_amount = 0.5
last_layer_dim = 10
filter_length = 8
conv_dim = 32
l2_lambda = 10**(-3.5)

def w_mse(weights):
    def loss(y_true, y_pred):
        return K.mean(K.sum(K.square(y_true-y_pred)*weights,axis=1)*tf.reshape(y_true[:,-1],(-1,1)))/c_i_factor
    return loss

#we don't want to optimize for the column counting video occurences, but
#they are included in the target so we can use that column for the loss function
weights = [1.0,0]

#fixed epoch budget of 100 that empirically seems to be sufficient 
n_epochs = 100

mse_opt = w_mse(weights)

#this is just for monitoring
mse_metric = w_mse(target_range**2*np.array(weights))

K.clear_session()

model = Sequential()
model.add(Conv1D(conv_dim,filter_length, input_dim=X_train.shape[2],input_length=vid_length,padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv1D(conv_dim,filter_length,padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(dropout_amount))
model.add(Flatten())
model.add(Dense(last_layer_dim,activation='relu'))
model.add(Dense(2, activation='linear'))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 8, padding="same", input_shape=(124, 12))`


In [22]:
checkpoint_folder = "./data/checkpoints/cnn_checkpoints_%s" % (target_col)

In [31]:
train_model = True

if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)

filepath=checkpoint_folder+"/weights-{epoch:02d}-{val_loss_1:.4f}.hdf5"
if train_model:

    opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(loss=mse_opt,metrics=[mse_metric],
                  optimizer=opt)


    checkpoint = \
        ModelCheckpoint(filepath, monitor='val_loss_1', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

    history = model.fit(X_train, y_train_scaled,callbacks=[checkpoint,lr,TerminateOnNaN()],
              validation_data=(X_validation,y_validation_scaled),
              batch_size=128, epochs=n_epochs,shuffle=True)

Train on 40852 samples, validate on 4938 samples
Epoch 1/100
40852/40852 [==============================] - 67s 2ms/step - loss: 0.0162 - loss_1: 94.2691 - val_loss: 0.0138 - val_loss_1: 82.7154

Epoch 00001: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-01-82.7154.hdf5
Epoch 2/100
40852/40852 [==============================] - 55s 1ms/step - loss: 0.0147 - loss_1: 90.0508 - val_loss: 0.0240 - val_loss_1: 150.8141

Epoch 00002: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-02-150.8141.hdf5
Epoch 3/100
40852/40852 [==============================] - 57s 1ms/step - loss: 0.0143 - loss_1: 88.1642 - val_loss: 0.0191 - val_loss_1: 119.2807

Epoch 00003: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-03-119.2807.hdf5
Epoch 4/100
40852/40852 [==============================] - 71s 2ms/step - loss: 0.0139 - loss_1: 85.4771 - val_loss: 0.0129 - val_loss_1: 79.1490

Epoch 00004: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-04-79.1

40852/40852 [==============================] - 51s 1ms/step - loss: 0.0105 - loss_1: 66.0494 - val_loss: 0.0132 - val_loss_1: 83.6029

Epoch 00035: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-35-83.6029.hdf5
Epoch 36/100
40852/40852 [==============================] - 48s 1ms/step - loss: 0.0105 - loss_1: 65.7593 - val_loss: 0.0126 - val_loss_1: 79.8000

Epoch 00036: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-36-79.8000.hdf5
Epoch 37/100
40852/40852 [==============================] - 46s 1ms/step - loss: 0.0104 - loss_1: 65.5873 - val_loss: 0.0111 - val_loss_1: 69.6769

Epoch 00037: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-37-69.6769.hdf5
Epoch 38/100
40852/40852 [==============================] - 49s 1ms/step - loss: 0.0104 - loss_1: 65.4839 - val_loss: 0.0116 - val_loss_1: 73.1240

Epoch 00038: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-38-73.1240.hdf5
Epoch 39/100
40852/40852 [===========================

40852/40852 [==============================] - 78s 2ms/step - loss: 0.0093 - loss_1: 58.7817 - val_loss: 0.0111 - val_loss_1: 70.3665

Epoch 00069: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-69-70.3665.hdf5
Epoch 70/100
40852/40852 [==============================] - 58s 1ms/step - loss: 0.0092 - loss_1: 58.3134 - val_loss: 0.0110 - val_loss_1: 70.0345

Epoch 00070: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-70-70.0345.hdf5
Epoch 71/100
40852/40852 [==============================] - 50s 1ms/step - loss: 0.0092 - loss_1: 58.0950 - val_loss: 0.0114 - val_loss_1: 72.1742

Epoch 00071: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-71-72.1742.hdf5
Epoch 72/100
40852/40852 [==============================] - 46s 1ms/step - loss: 0.0092 - loss_1: 58.0445 - val_loss: 0.0108 - val_loss_1: 68.7145

Epoch 00072: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-72-68.7145.hdf5
Epoch 73/100
40852/40852 [===========================

In [38]:
def undo_scaling(y,target_range,target_min):
    return y*target_range+target_min

In [44]:
weight_files = os.listdir(checkpoint_folder)
weight_files_df = pd.DataFrame(weight_files,columns=['filename'])
weight_files_df['num'] = weight_files_df['filename'].apply(lambda x: int(x.split('-')[1]))
weight_files_df.sort_values(by='num',ascending=True,inplace=True)

In [45]:
#todo: change so we also have a X,y for all_ids
#then for each of 100 epochs, generate the predictions of the model for each epoch
#this dataframe can then be saved, and a file called "get_best_epoch" can be used to generate the best one
#(video_id,target,predictions,dataset)

In [46]:
def predict_and_aggregate(y,X,ids,model,target_col):
    df = pd.DataFrame(y,columns=[target_col])
    side_col = ['L' if i < len(df)/2 else 'R' for i in range(len(df))]
    target_col_pred = target_col + "_pred"
    videoids = [t[0] for t in processed_video_segments if t[0] in ids]
    videoids = videoids + videoids
    df["videoid"] = np.array(videoids)
    df['side'] = side_col
    preds = model.predict(X)
    df[target_col_pred] = undo_scaling(preds[:,0],target_range,target_min)
    df["count"] = 1
    df = df.groupby(['videoid','side'],as_index=False).agg({target_col_pred:np.mean,'count':np.sum,target_col:np.mean})
    df['ones'] = 1
    return df

In [47]:
video_ids = [t[0] for t in processed_video_segments if t[0] in all_ids]
video_ids = video_ids + video_ids
predictions_df = pd.DataFrame(video_ids,columns=['videoid'])
side_col = ['L' if i < len(predictions_df)/2 else 'R' for i in range(len(predictions_df))]
predictions_df[target_col] = y
predictions_df['side'] = side_col
predictions_df = predictions_df.merge(right=datasplit_df[['videoid','dataset']],on=['videoid'],how='left')

In [48]:
for i in range(0,len(weight_files_df)):
    weight_file = weight_files_df['filename'].iloc[i]
    print(weight_file)
    model.load_weights(checkpoint_folder + "/%s" % (weight_file))
    preds = model.predict(X)
    predictions_df["%s_pred_%s" % (target_col,i)] = undo_scaling(preds[:,0],target_range,target_min)

weights-01-82.7154.hdf5
weights-02-150.8141.hdf5
weights-03-119.2807.hdf5
weights-04-79.1490.hdf5
weights-05-98.5648.hdf5
weights-06-137.9985.hdf5
weights-07-82.0750.hdf5
weights-08-91.3411.hdf5
weights-09-92.0958.hdf5
weights-10-74.3795.hdf5
weights-11-110.3506.hdf5
weights-12-79.4711.hdf5
weights-13-105.4354.hdf5
weights-14-94.0722.hdf5
weights-15-74.0778.hdf5
weights-16-86.4029.hdf5
weights-17-98.1366.hdf5
weights-18-89.0452.hdf5
weights-19-72.6494.hdf5
weights-20-93.7002.hdf5
weights-21-90.3839.hdf5
weights-22-93.8313.hdf5
weights-23-75.4599.hdf5
weights-24-83.4602.hdf5
weights-25-99.5875.hdf5
weights-26-75.6141.hdf5
weights-27-74.9093.hdf5
weights-28-82.5194.hdf5
weights-29-78.1027.hdf5
weights-30-75.9932.hdf5
weights-31-71.5938.hdf5
weights-32-78.7538.hdf5
weights-33-73.7134.hdf5
weights-34-73.9455.hdf5
weights-35-83.6029.hdf5
weights-36-79.8000.hdf5
weights-37-69.6769.hdf5
weights-38-73.1240.hdf5
weights-39-93.2331.hdf5
weights-40-74.9733.hdf5
weights-41-70.0237.hdf5
weights-42-

In [49]:
predictions_df.groupby(['videoid','side','dataset'],as_index=False).mean().to_csv("./data/predictions/cnn_%s_predictions_all_epochs.csv" % (target_col),index=False)